In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import altair as alt
import pandas as pd
import glob
import json
import os
import numpy as np
import math

from tensorboard.backend.event_processing import event_accumulator
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

alt.renderers.enable('notebook')

# job_regex = "bogo2*"
# job_dirs = glob.glob("/mnt/fair/code/neural-dynamics/results/" + job_regex)
basedir = '../'

def load_json(fname):
    with open(fname, 'r') as f:
        contents = json.load(f)
    return contents

def load_multiline_json(fname):
    with open(fname, 'r') as f:
        contents = [json.loads(line) for line in f]
    return contents

def load_jobs(regex):
    job_dirs = glob.glob(basedir + "results/" + regex)

    data_dict = {}
    for job in job_dirs:
        job_name = os.path.basename(os.path.normpath(job))
        options = load_json(job + '/opt.json')
        if os.path.exists(job + '/mpc_results.json'):
            results = load_json(job + '/mpc_results.json')
        else:
            continue
        job_data = {**options, **results, 'name': job_name}
        data_dict[job_name] = job_data

    return pd.DataFrame.from_dict(data_dict, orient='index')

def load_multiline_jobs(regex):
    job_dirs = glob.glob(basedir + "results/" + regex)

    rows = []
    option_keys = set()
    for job in job_dirs:
        job_name = os.path.basename(os.path.normpath(job))
        options = load_json(job + '/opt.json')
        if os.path.exists(job + '/mpc_results.json'):
            results = load_multiline_json(job + '/mpc_results.json')
        else:
            continue
        
        for result in results:
            rows.append({**options, **result})
        option_keys = option_keys | set(options.keys())

    data_dict = {i: rows[i] for i in range(len(rows))}

    df = pd.DataFrame.from_dict(data_dict, orient='index')
    return df, list(option_keys)

def load_mpc_sweep(regex):
    job_dirs = glob.glob(basedir + "results/" + regex)

    rows = []
    option_keys = set()
    for job in job_dirs:
        job_name = os.path.basename(os.path.normpath(job))
        options = load_json(job + '/opt.json')
        if os.path.exists(job + '/mpc_sweep.json'):
            results = load_multiline_json(job + '/mpc_sweep.json')
        else:
            continue
        
        for result in results:
            rows.append({**options, **result, 'name': job_name})
        option_keys = option_keys | set(options.keys())

    data_dict = {i: rows[i] for i in range(len(rows))}

    df = pd.DataFrame.from_dict(data_dict, orient='index')
    df['name'] = df['name'] + '_step' + df['plan_steps'].astype(str)
    if 'plan_cands' in df:
        df['name'] += '_cand' + df['plan_cands'].astype(str)
    return df, list(option_keys)

def load_current_data():
    csv_data = pd.read_csv("Swimmer MPC results - Raw data.csv")
    json_cem_data = load_jobs("cem_optim1*")
    other_json_data = load_jobs("cem_batched*")
    json_bogo_data = load_jobs("bogo2*")
    data = pd.concat([csv_data, json_cem_data, json_bogo_data, other_json_data], sort=True)
    return data

def sel_hist(brush, column):
    hist = alt.Chart().mark_bar().encode(
            y='{}:O'.format(column),
#             color='{}:O'.format(column),
            x='count({}):Q'.format(column)
        ).transform_filter(
            brush
        )
    return hist

def baselines():
    baseline_data = pd.DataFrame([
        {"mean_R": 110, "name": "PPO"},
        {"mean_R": 30, "name": "Berkeley"},
    ])

    rule = alt.Chart(baseline_data).mark_rule().encode(y='mean_R',)

    text = alt.Chart(baseline_data).mark_text(
        align='left', dx=-330, dy=-5
    ).encode(
        y='mean_R',
        text='name'
    )
    return rule + text

from tensorboard.backend.event_processing import event_accumulator
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

def load_tf(dirname, env="VisibleSwimmer-v2"):
    prefix = basedir + "tboard/{}/".format(env)
    dirname = prefix + dirname
    
#     print(dirname + '/events.out.tfevents*')
    matching_paths = reversed(sorted(glob.glob(dirname + '/events.out.tfevents*')))
    all_dframes = []
    for dirname in matching_paths:
#         print(dirname)

        ea = event_accumulator.EventAccumulator(dirname, size_guidance={event_accumulator.SCALARS: 0})
        ea.Reload()
        dframes = {}
        mnames = ea.Tags()['scalars']

        for n in mnames:
            dframes[n] = pd.DataFrame(ea.Scalars(n), columns=["wall_time", "epoch", n.replace('val/', '')])
            dframes[n].drop("wall_time", axis=1, inplace=True)
            dframes[n] = dframes[n].set_index("epoch")

        dframes = [v for k,v in dframes.items()]
        all_dframes += dframes

        if len(dframes) > 0:
            full_dframe = pd.concat(dframes, axis=1, sort=False)
            if 'train' in full_dframe:
                full_dframe['train_loss'] = full_dframe['train']
            all_dframes.append(full_dframe)
    if len(all_dframes) > 0:
        return pd.concat(all_dframes, sort=False)
    else:
        return None

def load_tf_jobs(regex, env="VisibleSwimmer-v2"):
    prefix = basedir + "results/"
    job_dirs = glob.glob(prefix + regex)

    rows = []
    for job in job_dirs:
#         print(job)
        job_name = os.path.basename(os.path.normpath(job))
        options = load_json(job + '/opt.json')
        results = load_tf(job.replace(prefix, ''), env=env)

        if results is not None:
            for opt in options:
                results[opt] = options[opt]
            rows.append(results)

    for row in rows:
        row['epoch'] = row.index
        row.reset_index(drop=True, inplace=True)
    df = pd.concat(rows, sort=False)
    df['trajectories'] = np.floor(df['epoch'] / df['aggregate_every']) * df['aggregate_steps']
    return df.reset_index()

def last_epoch(paths):
    last = -1
    for path in paths:
        try:
            epoch = int(os.path.split(os.path.dirname(os.path.normpath(path)))[1])
        except ValueError:
            continue
        last = epoch if epoch > last else last
    return last

def load_tf_rollout(dirname, env="VisibleSwimmer-v2", select_epoch=None):
    prefix = basedir + "tboard/{}/".format(env)
    dirname = prefix + dirname
    
    rollout_regex = dirname + '/*/events.out.tfevents*'
    matching_paths = list(reversed(sorted(glob.glob(rollout_regex))))
    all_dframes = []
    
    if select_epoch is None:
        select_epoch = last_epoch(matching_paths)
        
    if select_epoch < 0:
        last = last_epoch(matching_paths)
        select_epoch = last + select_epoch + 1
#     print(select_epoch)
    for dirname in matching_paths:
        try:
            epoch = int(os.path.split(os.path.dirname(os.path.normpath(dirname)))[1])
        except ValueError:
            continue
        if epoch != select_epoch:
            continue
#         print(epoch)

        ea = event_accumulator.EventAccumulator(dirname, size_guidance={event_accumulator.SCALARS: 0})
        ea.Reload()
        dframes = {}
        mnames = ea.Tags()['scalars']

        for n in mnames:
            dframes[n] = pd.DataFrame(ea.Scalars(n), columns=["wall_time", "depth", n.replace('rollout/', '')])
            dframes[n].drop("wall_time", axis=1, inplace=True)
            dframes[n] = dframes[n].set_index("depth")

        dframes = [v for k,v in dframes.items()]

        if len(dframes) > 0:
            full_dframe = pd.concat(dframes, axis=1, sort=False)
            full_dframe['epoch'] = epoch
            all_dframes.append(full_dframe)

    if len(all_dframes) > 0:
        return pd.concat(all_dframes, sort=False)
    else:
        return None

def load_tf_rollouts(regex, env="VisibleSwimmer-v2", select_epoch=None):
    prefix = basedir + "results/"
    job_dirs = glob.glob(prefix + regex)

    rows = []
    for job in job_dirs:
#         print(job)
        job_name = os.path.basename(os.path.normpath(job))
        options = load_json(job + '/opt.json')
        results = load_tf_rollout(job.replace(prefix, ''), env=env, select_epoch=select_epoch)

        if results is not None:
            for opt in options:
                results[opt] = options[opt]
            rows.append(results)

    df = pd.concat(rows, sort=False)
    return df.reset_index()

# load_tf_rollouts('rnntest', env="VisibleSwimmer-v2", select_epoch=-1)

/Users/willwhitney/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def impute(df):
    new_df = df.copy()
    if 'eval_step' in df.columns:
        steps_default = (df['eval_step'] - 1) * df['eval_freq'].astype(float)
        new_df['steps'].fillna(steps_default, inplace=True)
    return new_df

def impute_masking(df):
    new_df = pd.DataFrame()
#     df['mask_x'] = df['name'].str.contains('cem3') | df['mask_x']
#     df['mask_y'] = df['mask_y'].fillna(False)
    for _, row in df.iterrows():
        if not 'mask_x' in row:
            if 'cem3' in row['name']:
                row['mask_x'] = True
            else:
                row['mask_x'] = False
        if not 'mask_y' in row:
            row['mask_y'] = False
        new_df = new_df.append(row)
    return new_df

def drop_short_groups(df, groups, within=None, threshold_rel=1.0, threshold_abs=0):
    df = df.copy()
    if within is not None:
        within_values = df[within].unique()
        for within_value in within_values:
            grouped_df = df[df[within] == within_value].groupby(groups)
            max_len = 0
            for group in grouped_df.groups:
                max_len = max(max_len, len(grouped_df.get_group(group).index))
            for group in grouped_df.groups:
                index = grouped_df.get_group(group).index
                if len(index) < threshold_rel * max_len - threshold_abs:
                    df = df.drop(index)
    else:
        grouped_df = df.groupby(groups)
        max_len = 0
        for group in grouped_df.groups:
            max_len = max(max_len, len(grouped_df.get_group(group).index))
        for group in grouped_df.groups:
            index = grouped_df.get_group(group).index
            if len(index) < threshold_rel * max_len - threshold_abs:
                df = df.drop(index)
    return df

def cut(df, column, bins, groups=['name'], y_axis='mean_R', drop_short=True):
    binned_column = 'binned_' + column
    left_column = 'left_' + column
    right_column = 'right_' + column
    df = df.dropna(subset=[y_axis])
    df[binned_column] = pd.cut(df[column], bins)
    df[left_column] = df[binned_column].apply(lambda d: d.left)
    df[right_column] = df[binned_column].apply(lambda d: d.right)
    df = df.drop(binned_column, axis=1)
    if drop_short:
        df = drop_short_groups(df, ['name', left_column], threshold_abs=1)

    df = df.groupby(list({'name', 'seed', left_column, right_column, *groups}), as_index=False).mean()

    return df.dropna(subset=[y_axis])

def seed_stats(df, groups=['name'], y_axis='mean_R', drop_short=True):
    df['name'] = df['name'].str.replace('_seed[\d]+', '')

    if drop_short:
        df = drop_short_groups(df, groups, 'name', threshold_rel=0.75)
    for transform in ['mean', 'min', 'max', 'std', 'median']:
        df[transform + '_' + y_axis] = df.groupby(groups)[y_axis].transform(transform)
    
    df['seed_count'] = df.groupby(groups)['seed'].transform('count')
    df['seed_max'] = df.groupby('name')['seed_count'].transform(max)
    df['seed_frac'] = df['seed_count'] / df['seed_max']
#     df['seed_frac'] = df['seed_count'] / 8
    df = df.drop_duplicates(subset=groups)
    return df.dropna(subset=[y_axis])

def summarize_series(df, x_axis, bins, groups=['name'], y_axis='mean_R', 
                     collect_seeds=True, drop_short_bins=True, drop_short_seeds=True):
    df = df.copy()
    df = df.dropna(subset=[y_axis])
    groups = [group for group in groups if group != y_axis]
    df = cut(df, x_axis, bins, groups=groups, y_axis=y_axis, drop_short=drop_short_bins)
    if collect_seeds:
        df = seed_stats(df, groups=['left_' + x_axis, 'name'], y_axis=y_axis, drop_short=drop_short_seeds)
    else:
        df['seed_count'] = 1
        for transform in ['mean', 'min', 'max', 'median']:
            df[transform + '_' + y_axis] = df[y_axis]
    return df



In [4]:
def load_monitors(indir):
    datas = []
    infiles = glob.glob(os.path.join(indir, '*monitor.csv'))

    for inf in infiles:
#         print(inf)
        with open(inf, 'r') as f:
            log_line = f.readline()[1:]
            env = json.loads(log_line)['env_id']
            f.readline()
            for line in f:
                line = line.replace('\x00', '')
                tmp = line.split(',')
#                 print(tmp)
                t_time = float(tmp[2])
                tmp = [t_time, int(tmp[1]), float(tmp[0])]
                datas.append(tmp)

    datas = sorted(datas, key=lambda d_entry: d_entry[0])
#     print(datas)
    result = []
    timesteps = 0
    for i in range(len(datas)):
        result.append([i, timesteps, datas[i][1], datas[i][-1]])
        timesteps += datas[i][1]

#     if len(result) < bin_size:
#         return [None, None]

    result = np.array(result)
    if len(result) == 0:
        return None
    trajectories, steps, duration, reward = result[:, 0], result[:, 1], result[:, 2], result[:, 3]

    df = pd.DataFrame.from_dict(dict(zip(
        ['trajectories', 'steps', 'mean_duration', 'mean_R'], 
        [trajectories, steps, duration, reward]
    )))
    df['trajectories'] = df.index
    df['domain'] = env
    return df

def load_monitor_jobs(regex, basedir='../results/'):
    job_dirs = glob.glob(os.path.join(basedir, regex))

    rows = []
    for job in job_dirs:
        options = load_json(job + '/opt.json')
        job_name = os.path.basename(os.path.normpath(job))
#         print(job_name)
        results = load_monitors(job)
#         try:
#             results = load_monitors(job)
#         except:
#             results = None

        if results is not None:
#             print(results)
            options = {key: str(value) for (key, value) in options.items()}
            results = results.assign(**options)
#             for key, value in options.items():
#                 results = results.assign(key=options[key])
#                 print(results, key, options)
#             rows.append({**options, **results})
#             option_keys = option_keys | set(options.keys())


#             rows.append(results)


        if results is not None:
            rows.append(results)
    
    df = pd.concat(rows, sort=False)
    return df.reset_index(drop=True)

def load_evals(regex, basedir='../results/'):
    job_dirs = glob.glob(os.path.join(basedir, regex))

    rows = []
    for job in job_dirs:
        try:
            options = load_json(job + '/opt.json')
            job_name = os.path.basename(os.path.normpath(job))
            results = np.load(os.path.join(job, 'eval.npy'))
#             print(results)
            if len(results.shape) == 1:
                n = len(results)
                results = pd.DataFrame({'eval_step': np.linspace(1, n, n), 'mean_R': results})
            else:
                results = pd.DataFrame({'trajectories': results[:, 0], 'steps': results[:, 1], 'mean_R': results[:, 2]})
        except:
#             import traceback as tb; tb.print_exc()
            results = None
            
        if results is not None:
            options = {key: str(value) for (key, value) in options.items()}
            results = results.assign(**options)
            rows.append(results)

    df = pd.concat(rows, sort=False)
    return df.reset_index(drop=True)

# load_evals('SPM_nostart*', basedir='../../TD3/results/')[['name', 'trajectories', 'mean_R']]

In [ ]:
prefix = basedir + "tboard/{}/".format(env)
dirname = prefix + dirname

#     print(dirname + '/events.out.tfevents*')
matching_paths = reversed(sorted(glob.glob(dirname + '/events.out.tfevents*')))
all_dframes = []
for dirname in matching_paths:
#         print(dirname)

    ea = event_accumulator.EventAccumulator(dirname, size_guidance={event_accumulator.SCALARS: 0})
    ea.Reload()
    dframes = {}
    mnames = ea.Tags()['scalars']

    for n in mnames:
        dframes[n] = pd.DataFrame(ea.Scalars(n), columns=["wall_time", "epoch", n.replace('val/', '')])
        dframes[n].drop("wall_time", axis=1, inplace=True)
        dframes[n] = dframes[n].set_index("epoch")

    dframes = [v for k,v in dframes.items()]
    all_dframes += dframes

    if len(dframes) > 0:
        full_dframe = pd.concat(dframes, axis=1, sort=False)
        if 'train' in full_dframe:
            full_dframe['train_loss'] = full_dframe['train']
        all_dframes.append(full_dframe)
if len(all_dframes) > 0:
    return pd.concat(all_dframes, sort=False)
else:
    return None

